In [ ]:
import cv2
import numpy as np
from scipy.spatial import distance
import openai

In [ ]:
# OpenAI API 키 설정
openai.api_key = "your_api_key"

In [ ]:
# 1. 종양 마스크와 결과 계산 함수
def process_image(image_path):
    """
    입력 이미지를 처리하고, 마스크와 계산된 결과 반환
    """
    # 이미지 로드 (Grayscale)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # 1-1. 종양 중심 계산
    tumor_pixels = np.column_stack(np.where(image == 255))  # 흰색 픽셀 좌표
    tumor_center = tumor_pixels.mean(axis=0) if len(tumor_pixels) > 0 else (0, 0)

    # 1-2. 뇌 중심 계산
    brain_center = np.array(image.shape) / 2

    # 1-3. 거리 계산
    distance_to_center = distance.euclidean(tumor_center, brain_center)

    # 1-4. 종양 크기 계산
    tumor_size = len(tumor_pixels)

    # 1-5. 종양 불규칙성 계산 (컨투어 꼭짓점 수)
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    irregularity = len(cv2.convexHull(contours[0])) if contours else 0

    return image, tumor_center, brain_center, distance_to_center, tumor_size, irregularity


In [ ]:
# 2. 위험도 지수 계산 함수
def calculate_risk(distance, size, irregularity, weights):
    """
    위험도 지수를 계산
    """
    w1, w2, w3 = weights
    risk_score = w1 * distance + w2 * size + w3 * irregularity
    return risk_score

In [ ]:
gpt = "너는 ~~하는 애야."

In [ ]:
# 3. LLM 메시지 생성 함수
def llm_output(risk_score):
    """
    LLM을 사용하여 위험도 지수에 따른 메시지 출력
    """
    if risk_score <= 33:
        message = "종양의 위험도가 낮습니다. 정기적인 검진만 권장됩니다."
    elif risk_score <= 66:
        message = "종양의 위험도가 중간 수준입니다. 추가적인 검진이 필요합니다."
    else:
        message = "종양의 위험도가 높습니다. 즉각적인 의료 조치가 필요합니다."

    # LLM 메시지 생성
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a medical assistant."},
            {"role": "user", "content": f"위험도: {risk_score}\n{message}"}
        ]
    )
    return response['choices'][0]['message']['content']

In [ ]:
# 4. 전체 파이프라인 실행
def main(image_path, weights):
    # 이미지 처리 및 결과 계산
    image, tumor_center, brain_center, distance_to_center, tumor_size, irregularity = process_image(image_path)

    # 위험도 지수 계산
    risk_score = calculate_risk(distance_to_center, tumor_size, irregularity, weights)

    # LLM 메시지 출력
    llm_message = llm_output(risk_score)

    # 결과 출력
    print("=== 결과 ===")
    print(f"종양 중심: {tumor_center}")
    print(f"뇌 중심: {brain_center}")
    print(f"종양-뇌 중심 거리: {distance_to_center}")
    print(f"종양 크기(픽셀 수): {tumor_size}")
    print(f"종양 모양 불규칙성(꼭짓점 수): {irregularity}")
    print(f"위험도 지수: {risk_score}")
    print("\nLLM 메시지:")
    print(llm_message)

    # 이미지와 마스크 시각화
    cv2.imshow("Tumor Mask", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# 실행 예제
if __name__ == "__main__":
    # 테스트용 이미지 경로와 가중치 설정
    test_image_path = "masked_image.png"  # 예시 이미지 경로
    risk_weights = (0.3, 0.5, 0.2)  # 가중치 (거리, 크기, 불규칙성)

    main(test_image_path, risk_weights)